# Word2Vec from Scratch
(by Tevfik Aytekin)

In [6]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import gutenberg, brown
import gensim 
from gensim.models import Word2Vec 
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.utils import shuffle
from queue import PriorityQueue


# You need to call nltk.download() to download all the nltk corpora

## Definition from Wikipedia:

“Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space.”

In [7]:
nltk.download('brown')
num_sents = len(brown.sents())
print("number of sentences:", num_sents)

[nltk_data] Downloading package brown to
[nltk_data]     /Users/tevfikaytekin/nltk_data...
[nltk_data]   Package brown is already up-to-date!


number of sentences: 57340


An example sentence represented as a list of words

In [8]:
brown.sents()[0]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

Following is an example application of word2vec using Gensim library. You can see some of the parameters and can find all the details of Gensim implementation [here](https://radimrehurek.com/gensim/models/word2vec.html). The Gensim word2vec source code is [here](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/word2vec.py) and the original source code by Mikolov can be found [here](https://github.com/tmikolov/word2vec/blob/master/word2vec.c).



In [9]:
model = gensim.models.Word2Vec(brown.sents(),min_count = 5,
                              size = 30, window = 5, iter=5, negative=5) 

An example vector representation of the word "book". Since we set size = 30, the representation is an array of 30 reals.

In [10]:
model.wv['book']

array([-0.7311977 ,  0.31063437,  0.6791632 , -0.9563669 ,  1.0403818 ,
        0.1184039 , -0.33935213,  0.42266575, -0.22066063, -0.5223206 ,
        0.68718785,  0.19647512,  0.5603375 , -0.20316611,  0.5216088 ,
       -0.67623955, -0.24772866, -0.10938344,  0.00322819, -0.5232904 ,
       -0.5702316 ,  0.52905756, -0.2803416 ,  0.02922845, -0.9915429 ,
        0.41968042, -0.9494648 ,  1.0896757 , -0.11942578, -1.2495725 ],
      dtype=float32)

One way to test the performance of word2vec is to look at most similar words to a given word. Below you will find most similar words of the words "book" and "eight"

In [11]:
model.wv.most_similar(positive='book')

[('story', 0.9458640217781067),
 ('opinion', 0.9273183345794678),
 ('gapt', 0.9207190871238708),
 ('era', 0.9130477905273438),
 ('novel', 0.9127198457717896),
 ('hero', 0.9124677181243896),
 ('statement', 0.9094814658164978),
 ('workmanship', 0.9092841148376465),
 ('artist', 0.9090732336044312),
 ('fellow', 0.9089953303337097)]

In [12]:
model.wv.most_similar(positive='eight')

[('seven', 0.9732145667076111),
 ('thirty', 0.9490191340446472),
 ('fifteen', 0.9430728554725647),
 ('eleven', 0.942838191986084),
 ('decades', 0.9411295652389526),
 ('nine', 0.9338229894638062),
 ('fifty', 0.9309582710266113),
 ('twelve', 0.929398775100708),
 ('seasons', 0.9271189570426941),
 ('prison', 0.9263835549354553)]

As you can see the results are quite amazing. But it might not be so for every word, for example for the word "angry" the results are not very satisying. However, if have used a larger text the results could be better.

In [13]:
model.wv.most_similar(positive='angry')

[('ankle', 0.9636025428771973),
 ('released', 0.9600746631622314),
 ('waking', 0.9585892558097839),
 ('suffering', 0.9565783739089966),
 ('delinquent', 0.9563600420951843),
 ('guilt', 0.9555759429931641),
 ('master', 0.9548867344856262),
 ('minister', 0.9537694454193115),
 ('gift', 0.9533553123474121),
 ('dressed', 0.9529207944869995)]

You can also find (cosine) similarity between two words.

In [14]:
print("Cosine similarity between 'book' and 'story':", 
    model.wv.similarity('book', 'story')) 

Cosine similarity between 'book' and 'story': 0.9458641


In [15]:
print("Cosine similarity between 'book' and 'eight':", 
    model.wv.similarity('book', 'eight')) 

Cosine similarity between 'book' and 'eight': 0.50872284


## word2vec from scratch

Now we will write word2vec from scratch. Note that the purpose of this implementation is to help understand the theory behind word2vec. The implementation is not meant to be efficient so the running time is quite slow compared to the Gensim implementation. However, the code is simpler and shows the main ingredients of word2vec.

Different objectives can be used for word2vec. The following is the objective for word2vec with negative sampling. The main idea behind this objective is to find paramater values which maximizes the dot product of word representations which are in the same context and minimizes the dot product of word representations which are not in the same context.


$$
J(\Theta) = \underset{\theta}{\operatorname{argmax}}{\sum_{c,t \in D_p}log(\sigma(v_c \cdot v_t))+\sum_{c,t \in D_n}log(\sigma(-v_c \cdot v_t))}
$$

Here, $D_p$ is the set of word pairs whose distance is at most $m$ and $D_n$ is the set of unrelated (negative) word pairs, i.e., word pairs whose distance is larger than $m$, and $\sigma$ is the sigmoid function. Below we find the derivative of this function with respect to positive and negative words which we will use in the updates of gradient descent algorithm.
$$
\frac{\partial J(\Theta)}{\partial v_{c}}=\sum_{c,t \in D_p}\frac{1}{\sigma(v_c \cdot v_t)}\sigma(v_c \cdot v_t)(1-\sigma(v_c \cdot v_t))(v_t)\\
+ \sum_{c,t \in D_n}\frac{1}{\sigma(-v_c \cdot v_t)}\sigma(-v_c \cdot v_t)(1-\sigma(-v_c \cdot v_t))(-v_t)\\
= \sum_{c,t \in D_p}(1-\sigma(v_c \cdot v_t))v_t + \sum_{c,t \in D_n}-(1-\sigma(-v_c \cdot v_t))v_t 
$$
$$
\frac{\partial J(\Theta)}{\partial v_{t \in D_p}}=\sum_{c,t \in D_p}(1-\sigma(v_c \cdot v_t))v_c 
$$
$$
\frac{\partial J(\Theta)}{\partial v_{t \in D_n}}=\sum_{c,t \in D_n}-(1-\sigma(-v_c \cdot v_t))v_c 
$$

In [16]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))  

In [17]:
def build_indices(sents):
    """ 
  
    Parameters: 
    sents: A list of sentecens and each sentence is a list of words (i.e., a list of lists). 
  
    Returns: 
    word_freqs: frequency of each word
    word_to_index: a mapping from word names to integers.
    index_to_word: a mapping from integers to word names.
    
  
    """
    counter = 0
    word_freqs = {}
    word_to_index = {}
    index_to_word = {}
    for i in range(len(sents)): 
        for j in range(len(sents[i])):
            w = sents[i][j].lower()
            if w in word_freqs:
                word_freqs[w] += 1
            else:
                word_freqs[w] = 1
                word_to_index[w] = counter
                index_to_word[counter] = w
                counter += 1
            
    return word_freqs, word_to_index, index_to_word

In [18]:
def build_training_set(sents, word_freqs, window=5, sampling_freq = 0.001, neg_exp = 0.75, num_negs = 1, min_count=5):
    """ 
    Builds a trainig set
    
    Parameters: 
    sents: A list of sentecens and each sentence is a list of words (i.e., a list of lists).
    word_freqs: Frequency of words.
    windows: size of the context window.
    sampling_freq: words whose frequency larger than this value will be discarded.
    neg_exp: used for adjusting the negative sampling distribution.
    window: The size of the context window.
  
    Returns: 
    training_set: list of context word, positive and negatives
    """
    words_list = []
    total_freq = sum(word_freqs.values())
    
    #total_freq = sum([freq**(neg_exp) for freq in word_freqs.values()])
    word_array = []
    for word, freq in word_freqs.items():
        if ((word_freqs[word]/total_freq) < sampling_freq) and (word_freqs[word] > min_count):
            words_list.append(word)
            for i in range(int(freq**neg_exp)):
                word_array.append(word)
    
    training_set = []
    
    sampled_sents = []
    for i in range(len(sents)): 
        sent = []
        for j in range(len(sents[i])):
            w = sents[i][j].lower()
            if ((word_freqs[w] / total_freq) < sampling_freq) and (word_freqs[w] > min_count):
                sent.append(w)
        sampled_sents.append(sent)
    
    
    for i in range(len(sampled_sents)): 
 
        for j, w in enumerate(sampled_sents[i]):
            context = []
            for k in range(max(j-window,0),min(j+window+1,len(sampled_sents[i]))):
                w_p = sampled_sents[i][k]
                if (w == w_p):
                    continue
                w_n = []
                for k in range(num_negs):
                    w_n.append(word_array[np.random.randint(0,len(word_array))] )
                training_set.append([w,w_p,w_n])

    return training_set, np.unique(words_list)

In order to understand the produced training_set here is a very simple example sentence consisting of 6 words.

In [19]:
sents = [["a","b","c","d","e","f"]]
word_freqs = {"a":1,"b":1,"c":1,"d":1,"e":1,"f":1}


In [20]:
training_set, words_list = build_training_set(sents,window=1, word_freqs= word_freqs , sampling_freq = 1, min_count= 0)

In [21]:
# print training set
training_set

[['a', 'b', ['c']],
 ['b', 'a', ['a']],
 ['b', 'c', ['f']],
 ['c', 'b', ['a']],
 ['c', 'd', ['d']],
 ['d', 'c', ['f']],
 ['d', 'e', ['d']],
 ['e', 'd', ['f']],
 ['e', 'f', ['c']],
 ['f', 'e', ['f']]]

Let us now build the training set for the brown dataset which will take some time

In [22]:
word_freqs, word_to_index, index_to_word = build_indices(brown.sents())

In [23]:
training_set, words_list = build_training_set(brown.sents(),word_freqs)

In [24]:
# print first 10 examples in the trainigng set
training_set[:10]

[['fulton', 'county', ['relations']],
 ['fulton', 'grand', ["don't"]],
 ['fulton', 'jury', ['existence']],
 ['fulton', 'friday', ['replaced']],
 ['fulton', 'investigation', ["'"]],
 ['county', 'fulton', ['emerge']],
 ['county', 'grand', ['assigned']],
 ['county', 'jury', ['towne']],
 ['county', 'friday', ['still']],
 ['county', 'investigation', ['members']]]

In [25]:
def build_model(training_set, initial_alpha = 0.025, min_alpha = 0.0001, n_iters = 5, my_lambda = 0, vector_size = 30):
    word_vectors = {}
    
    # initialize word vectors
    for n in range(len(words_list)):
        word_vectors[words_list[n]] = np.random.rand(vector_size,1) - 0.5
    

    alpha = initial_alpha
    for t in range(n_iters):
        training_set = shuffle(training_set)
        objective = 0
        for ex in training_set:
            w = ex[0]
            w_p = ex[1]
            w_n_list = ex[2]
            w_v = word_vectors[w]
            w_p_v = word_vectors[w_p]
            word_vectors[w_p] = w_p_v + alpha*(((1-sigmoid(np.dot(w_v.T,w_p_v)))*w_v)-my_lambda*w_p_v)
            objective += np.log((sigmoid(np.dot(w_v.T,w_p_v))))

            for n in range(len(w_n_list)):
                w_n = w_n_list[n]
                w_n_v = word_vectors[w_n]
                word_vectors[w_n] = w_n_v + alpha*((-(1-sigmoid(-np.dot(w_v.T,w_n_v)))*w_v)-my_lambda*w_n_v)      
                objective += np.log((sigmoid(-np.dot(w_v.T,w_p_v))))
     
        alpha = initial_alpha - ((initial_alpha - min_alpha) * t / n_iters)
        print("alpha: ",alpha)
        print("Iteration: ", t)
        print("Objective: ", objective)
        print("cosine of words 'friend' and 'fellow': ",np.dot(word_vectors['friend'].T, word_vectors['fellow']))
    return word_vectors

In [26]:
word_vectors = build_model(training_set, initial_alpha = 0.025, min_alpha = 0.0001, n_iters = 5, my_lambda = 0, vector_size = 30)

alpha:  0.025
Iteration:  0
Objective:  [[-4626853.52196624]]
cosine of words 'friend' and 'fellow':  [[-0.04599253]]
alpha:  0.020020000000000003
Iteration:  1
Objective:  [[-4763095.84161801]]
cosine of words 'friend' and 'fellow':  [[0.07913852]]
alpha:  0.015040000000000001
Iteration:  2
Objective:  [[-4902565.08121694]]
cosine of words 'friend' and 'fellow':  [[0.3549099]]
alpha:  0.010060000000000001
Iteration:  3
Objective:  [[-4988008.4960619]]
cosine of words 'friend' and 'fellow':  [[0.59710753]]
alpha:  0.005080000000000001
Iteration:  4
Objective:  [[-5038439.56381079]]
cosine of words 'friend' and 'fellow':  [[0.67962624]]


In [27]:
def most_similar(word, word_vectors):
    pq = PriorityQueue()
    for w in word_vectors.keys():
        pq.put((-np.dot(word_vectors[word].T, word_vectors[w]), w))
    return pq

In [28]:
pq = most_similar('book', word_vectors)

In [29]:
for i in range(10):
    print(pq.get())

(array([[-3.04192531]]), 'book')
(array([[-2.12554841]]), 'mr.')
(array([[-2.0974355]]), 'john')
(array([[-2.02272618]]), 'partisan')
(array([[-1.97457834]]), 'james')
(array([[-1.96470536]]), 'presented')
(array([[-1.87226182]]), 'mason')
(array([[-1.86870251]]), 'treason')
(array([[-1.86716308]]), 'b.')
(array([[-1.86235148]]), 'william')


In [397]:
pq = most_similar('eight', word_vectors)
for i in range(10):
    print(pq.get())

(array([[-3.42139827]]), 'eight')
(array([[-2.88453067]]), 'approximately')
(array([[-2.47945924]]), 'per')
(array([[-2.38159085]]), 'hundred')
(array([[-2.36105726]]), '30')
(array([[-2.34564456]]), 'million')
(array([[-2.31762669]]), 'ten')
(array([[-2.2468595]]), '25')
(array([[-2.22857262]]), '100')
(array([[-2.19047865]]), 'june')


### Some questions
- Even though "opinion" and "story" does not appear in the context of "book" (window size 5) still word2vec puts these close to book, which is remarkable. Can you find this by building a co-occurrence matrix?
